In [7]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from skimage.color import gray2rgb

import matplotlib.pyplot as plt

import sarpy as sp
from sarpy.datasets import load_mpeg7

import keras
import keras.applications.inception_v3 as inception
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

In [2]:
def get_class_name(target):
    if target == 0: return "Bone"
    if target == 1: return "Comma"
    if target == 2: return "Glas"
    if target == 3: return "HCircle"
    if target == 4: return "Heart"
    if target == 5: return "Misk"
    if target == 6: return "apple"
    if target == 7: return "bat"
    if target == 8: return "beetle"
    if target == 9: return "bell"
    if target == 10: return "bird"
    if target == 11: return "bottle"
    if target == 12: return "brick"
    if target == 13: return "butterfly"
    if target == 14: return "camel"
    if target == 15: return "car"
    if target == 16: return "carriage"
    if target == 17: return "cattle"
    if target == 18: return "cellular_phone"
    if target == 19: return "chicken"
    if target == 20: return "children"
    if target == 21: return "chopper"
    if target == 22: return "classic"
    if target == 23: return "crown"
    if target == 24: return "cup"
    if target == 25: return "deer"
    if target == 26: return "device0"
    if target == 27: return "device1"
    if target == 28: return "device2"
    if target == 29: return "device3"
    if target == 30: return "device4"
    if target == 31: return "device5"
    if target == 32: return "device6"
    if target == 33: return "device7"
    if target == 34: return "device8"
    if target == 35: return "device9"
    if target == 36: return "dog"
    if target == 37: return "elephant"
    if target == 38: return "face"
    if target == 39: return "fish"
    if target == 40: return "flatfish"
    if target == 41: return "fly"
    if target == 42: return "fork"
    if target == 43: return "fountain"
    if target == 44: return "frog"
    if target == 45: return "guitar"
    if target == 46: return "hammer"
    if target == 47: return "hat"
    if target == 48: return "horse"
    if target == 49: return "horseshoe"
    if target == 50: return "jar"
    if target == 51: return "key"
    if target == 52: return "lizzard"
    if target == 53: return "lmfish"
    if target == 54: return "octopus"
    if target == 55: return "pencil"
    if target == 56: return "personal_car"
    if target == 57: return "pocket"
    if target == 58: return "rat"
    if target == 59: return "ray"
    if target == 60: return "sea_snake"
    if target == 61: return "shoe"
    if target == 62: return "spoon"
    if target == 63: return "spring"
    if target == 64: return "stef"
    if target == 65: return "teddy"
    if target == 66: return "tree"
    if target == 67: return "truck"
    if target == 68: return "turtle"
    if target == 69: return "watch"

In [16]:
# Loading dataset
dataset = load_mpeg7()
bitmaps, targets = dataset["bitmaps"][:140], dataset["targets"][:140]

# Normalizing images
image_size_height = image_size_width = 299
#bitmaps = [bitmap.normalize(image_size_height,image_size_width) for bitmap in bitmaps]
bitmaps = [image.img_to_array(gray2rgb(resize(bitmap.data, (299,299), mode="constant"))) for bitmap in bitmaps]
targets = keras.utils.to_categorical(targets, num_classes=7)

In [17]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(bitmaps, targets, test_size = 0.5, random_state=42)


[ 0.  0.  0.  0.  1.  0.  0.]


In [19]:
# create the base pre-trained model, without top
base_model = inception.InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
# Fully-connected layer
x = Dense(1024, activation='relu')(x)
# Classes logistic layer -- with 70 classes
predictions = Dense(7, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit(x=np.array(X_train),y=np.array(y_train),batch_size=32, epochs=1)

ValueError: Error when checking target: expected dense_10 to have 4 dimensions, but got array with shape (70, 7)